# Walkable City Project

Problem:
I have received a job offer in Seattle and intend to move there, but I am not sure of the neighborhood that is best for me.  I currently live in the downtown area of Chicago and I would like to compare my neighborhood to a similar neighborhood in Seattle.  This would include walkability to grocery stores, proximity to restaurants and bars and access to public transportation.  Criteria will include number of grocery stores in neighborhood, the number of restaurants and bars in a neighborhood and the distance from the center of neighborhood to the closest public transit station, trains not buses.  

Data Needed:
The data needed will be the neighborhood names and zip codes for each of the cities (Chicago and Seattle) as well as the latitude and longitude data for the center of each of those neighborhoods.  Using that information, data will be downloaded from FourSquare to access grocery stores, restaurants (including coffee shops), bars and public transportation stops.  A DataFrame of all of the neighborhoods will be constructed with each of those neighborhoods and a similarity score will be determined with Cosine Similarity to compare it to my Chicago neighborhood.  After finding the neighborhood most similar, I will create clusters of venues based on lat/long data to find the location of the best place to live near a venue dense area and plot them on a map.
